# Notebook for learning from data coursework 2022

In [19]:
# import our libraries that we will be using to investigate the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## load out csv data into a pandas dataframe so that we can manipulate it

In [20]:
dataframe = pd.read_csv("AT_T_SBC_Stock_Price_Data_2000_2020.csv")
print(dataframe)

      a_permno  PERMNO      DATE     CUSIP    NCUSIP  \
0        66093   66093  20000103  00206R10  78387G10   
1        66093   66093  20000104  00206R10  78387G10   
2        66093   66093  20000105  00206R10  78387G10   
3        66093   66093  20000106  00206R10  78387G10   
4        66093   66093  20000107  00206R10  78387G10   
...        ...     ...       ...       ...       ...   
5282     66093   66093  20201224  00206R10  00206R10   
5283     66093   66093  20201228  00206R10  00206R10   
5284     66093   66093  20201229  00206R10  00206R10   
5285     66093   66093  20201230  00206R10  00206R10   
5286     66093   66093  20201231  00206R10  00206R10   

                        COMNAM TICKER  PERMCO  SHRCD  SHRCLS  ...     NAICS  \
0     S B C COMMUNICATIONS INC    SBC   21645     11     NaN  ...       NaN   
1     S B C COMMUNICATIONS INC    SBC   21645     11     NaN  ...       NaN   
2     S B C COMMUNICATIONS INC    SBC   21645     11     NaN  ...       NaN   
3     S B C

In [21]:
# load our data into a pandas dataframe

# create our time series data
time_series_depth = 5

# construct the input data to the machine learning model

for i in range(time_series_depth):
    date_col = []
    prc_col = []
    open_prc_col = []
    askhi_col = []
    bidlo_col = []
    vol_col = []
    for j in range(len(dataframe)):
        date_col.append(dataframe.iloc[[j - i]]["DATE"])
        prc_col.append(dataframe.iloc[[j - i]]["PRC"])
        open_prc_col.append(dataframe.iloc[[j - i]]["OPENPRC"])
        askhi_col.append(dataframe.iloc[[j - i]]["ASKHI"])
        bidlo_col.append(dataframe.iloc[[j - i]]["BIDLO"])
        vol_col.append(dataframe.iloc[[j - i]]["VOL"])
    dataframe['step' + str(i) + 'DATE'] = date_col
    dataframe['step' + str(i) + 'PRC'] = prc_col
    dataframe['step' + str(i) + 'OPENPRC'] = open_prc_col
    dataframe['step' + str(i) + 'ASKHI'] = askhi_col
    dataframe['step' + str(i) + 'BIDLO'] = bidlo_col
    dataframe['step' + str(i) + 'VOL'] = vol_col


# we cant include the data for the stock we are predicting
#col_selection = ["DATE", "OPENPRC", "PRC", "ASKHI", "BIDLO", "VOL"]
col_selection = []

for i in range(time_series_depth):
    col_selection.append("step" + str(i) + "DATE")
    col_selection.append("step" + str(i) + "OPENPRC")
    col_selection.append("step" + str(i) + "PRC")
    col_selection.append("step" + str(i) + "ASKHI")
    col_selection.append("step" + str(i) + "BIDLO")
    col_selection.append("step" + str(i) + "VOL")

X = dataframe[col_selection]
Y = dataframe["PRC"]
    
reg = LinearRegression()

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

reg.fit(x_train, y_train)

predictions = reg.predict(x_test)
total_error = 0
for i in range(len(predictions)):
    total_error += pow(predictions[i] - y_test.array[i], 2)

print("total error squared: ", total_error)

      a_permno  PERMNO      DATE     CUSIP    NCUSIP  \
0        66093   66093  20000103  00206R10  78387G10   
1        66093   66093  20000104  00206R10  78387G10   
2        66093   66093  20000105  00206R10  78387G10   
3        66093   66093  20000106  00206R10  78387G10   
4        66093   66093  20000107  00206R10  78387G10   
...        ...     ...       ...       ...       ...   
5282     66093   66093  20201224  00206R10  00206R10   
5283     66093   66093  20201228  00206R10  00206R10   
5284     66093   66093  20201229  00206R10  00206R10   
5285     66093   66093  20201230  00206R10  00206R10   
5286     66093   66093  20201231  00206R10  00206R10   

                        COMNAM TICKER  PERMCO  SHRCD  SHRCLS  ...  \
0     S B C COMMUNICATIONS INC    SBC   21645     11     NaN  ...   
1     S B C COMMUNICATIONS INC    SBC   21645     11     NaN  ...   
2     S B C COMMUNICATIONS INC    SBC   21645     11     NaN  ...   
3     S B C COMMUNICATIONS INC    SBC   21645     1